# Promethee I Ranking

This module shows an example of how to use the "Promethee I Ranking" module

In [1]:
%config Completer.use_jedi = False

## Definition of inputs

In [2]:
import pandas as pd
import sys
sys.path.append('../..')

from modular_parts.ranking import calculate_prometheeI_ranking

## Problem formalization

In [3]:
alternatives = ['a1', 'a2', 'a3', 'a4', 'a5', 'a6']

In [4]:
outranking_flows = pd.DataFrame({'positive': [0.04151, 0.12188, 0.26870, 0.60934, 0.26058, 0.03652],
                                 'negative': [0.49889, 0.22204, 0.08527, 0.05025, 0.08933, 0.39276]}, index=alternatives)

## Promethee I Ranking

In [5]:
promethee_rank = calculate_prometheeI_ranking(outranking_flows, weak_preference=False)
promethee_rank

,a1,a2,a3,a4,a5,a6
a1,None,?,?,?,?,?
a2,P,None,?,?,?,P
a3,P,P,None,?,P,P
a4,P,P,P,None,P,P
a5,P,P,?,?,None,P
a6,?,?,?,?,?,None


With weak_preference parameter

In [6]:
promethee_rank2 = calculate_prometheeI_ranking(outranking_flows, weak_preference=True)
promethee_rank2

,a1,a2,a3,a4,a5,a6
a1,None,?,?,?,?,?
a2,S,None,?,?,?,S
a3,S,S,None,?,S,S
a4,S,S,S,None,S,S
a5,S,S,?,?,None,S
a6,?,?,?,?,?,None
